In [2]:
import json

import pandas as pd
import numpy as np

from scipy.sparse import lil_matrix

import matplotlib.pyplot as plt
actor_name_map = {}
movie_actor_map = {}
actor_genre_map = {}


with open("movie_data.csv", "r") as in_file:
    for line in in_file:
        
        this_movie = json.loads(line)
        
        if len(this_movie["rating"]) == 0:
            continue
                    
        for actor_id,actor_name in this_movie['actors']:
            actor_name_map[actor_id] = actor_name
            
        for actor_id,actor_name in this_movie['actors']:
            this_actors_genres = actor_genre_map.get(actor_id, {})
            
            for g in this_movie["genres"]:
                this_actors_genres[g] = this_actors_genres.get(g, 0) + 1
                
            actor_genre_map[actor_id] = this_actors_genres
            
        movie_actor_map[this_movie["imdb_id"]] = ({
            "movie": this_movie["title"],
            "actors": set([item[0] for item in this_movie['actors']]),
            "genres": this_movie["genres"],
            "rating": this_movie["rating"]["avg"]
        })
print("Known Actors:", len(actor_name_map))
print("Known Movies:", len(movie_actor_map))

actor_id_to_index = {actor_id:i for i,actor_id in enumerate(actor_name_map.keys())}

target_movie_id = "tt0317705" # The Incredibles
target_movie_id = "tt0816692" # Interstellar
target_movie_id = "tt0332280" # The Notebook

target_movie_object = movie_actor_map[target_movie_id]
target_movie_object

movie_similarities = []

for this_movie_id,this_movie_obj in movie_actor_map.items():
    if this_movie_id == target_movie_id:
        continue
        
    this_intersect = this_movie_obj["actors"].intersection(target_movie_object["actors"])
    this_union = this_movie_obj["actors"].union(target_movie_object["actors"])
    
    jaccard = len(this_intersect) / len(this_union)
    
    movie_similarities.append({
        "movie": this_movie_id,
        "jaccard": jaccard,
    })
similarity_df = pd.DataFrame(movie_similarities, columns=["movie", "jaccard"])

k_nn = 5
similarity_df.sort_values(by="jaccard", ascending=False).head(k_nn)

top_movies = similarity_df.sort_values(by="jaccard", ascending=False).head(k_nn)

relevant_genre_count = {}
for this_movie_id in top_movies["movie"]:
    print(this_movie_id, movie_actor_map[this_movie_id]["movie"])
    print("\t", movie_actor_map[this_movie_id]["genres"])
    
    this_movie_genres = movie_actor_map[this_movie_id]["genres"]
    for g in this_movie_genres:
        relevant_genre_count[g] = relevant_genre_count.get(g, 0) + 1
        
#As we increase k, the inferred genre changes slightly with each k value increase. The movie after the initial movie has one similar genre, and the one after that will have one similar genre with the movie before it.




JSONDecodeError: Expecting value: line 1 column 1 (char 0)